In [1]:
import stanfordnlp
stanfordnlp.download('en')
nlp = stanfordnlp.Pipeline(processors = "tokenize,mwt,lemma,pos")
doc = nlp("Event Management is the process that monitors all events that occur through the IT infrastructure to allow for normal operation and also to detect and escalate exception conditions.")
doc.sentences[0].print_tokens()


Using the default treebank "en_ewt" for language "en".
Would you like to download the models for: en_ewt now? (Y/n)

Default download directory: C:\Users\User\stanfordnlp_resources
Hit enter to continue or type an alternate directory.

Download location: C:\Users\User\stanfordnlp_resources\en_ewt_models.zip


  0%|          | 0.00/235M [00:00<?, ?B/s]


KeyboardInterrupt: 

In [3]:
print(*[f'text: {word.text+" "}\tlemma: {word.lemma}\tupos: {word.upos}\txpos: {word.xpos}' for sent in doc.sentences for word in sent.words], sep='\n')




NameError: name 'doc' is not defined

In [312]:
with open("ITIL Books/ITIL 3/Service operation glossary/definitions.txt") as file:
    glossary_definitions = file.read()

# definitions = [i.lower().replace("\n", " ").split('::') for i in glossary_definitions.split('\n\n')]
definitions = [i.replace("\n", " ").split('::') for i in glossary_definitions.split('\n\n')]

concepts_from_definitions = [i[0].strip() for i in definitions]

with open("ITIL Books/ITIL 3/Service operation chapter 4/Service operation chapter 4 - all.txt") as file:
    text = file.read()

ontology = {}

for index in range(0, len(definitions)):
    ontology[definitions[index][0].strip()] = [f"definition::{definitions[index][1].strip()}"]

import nltk
# Tokenize into sentences
sentences = nltk.sent_tokenize(text)

# Split sentences into words
# sentences = [nltk.word_tokenize(i.lower().replace("\n", " ")) for i in sentences]
sentences = [nltk.word_tokenize(i.replace("\n", " ")) for i in sentences]

sentences_containing_concepts = {}
for sentence in sentences:
    # print("---")
    count = 0
    for concept in concepts_from_definitions:
        concept_list = concept.split()
        
        if concept_list[0] in sentence:
            add_to_sentences_containing_concepts = True
            
            if len(concept_list) > 1:
                current_word_index = sentence.index(concept_list[0])
                concept_word_length_count = 1
                for concept_word in concept_list[1::]:
                    current_word_index = current_word_index + 1
                    
                    if sentence[current_word_index] != concept_word:
                        add_to_sentences_containing_concepts = False
                        break
                
            if add_to_sentences_containing_concepts is True:
                sentence_string = " ".join(sentence)
                if sentence_string not in sentences_containing_concepts.keys():
                    sentences_containing_concepts[sentence_string] = []
                sentences_containing_concepts[sentence_string].append(concept)
            

# sentences_containing_concepts = [i for i in sentences if i in concepts_from_definitions]
 
# for sentence in sentences_containing_concepts:
#     print(sentence)

# print(sentences_containing_concepts)

# print(ontology)

for sentence, concepts in sentences_containing_concepts.items():
    for concept in concepts:
        ontology[concept].append(sentence)
        
# TODO: In definitions replace (book name) with own in book relation


In [315]:
import re
import random

def load_templates(file_path):
    with open(file_path) as template_file:
        templates = template_file.readlines()
    return templates

# question_template = "question_template(Question_number=1,Answer_number=4)::Question->What is the definition for {concept.name}?::Answer->{concept.definition}"
templates = load_templates("templates/question_templates.txt")
question_template = templates[random.randint(0, len(templates) - 1)]
# question_template = templates[1]

# print(templates)
# print(question_template)

# nltk.tokenize.mwe 
        
def get_definition(concept_name):
    return [i.split("::")[1] for i in ontology[concept_name] if i.split("::")[0] == "definition"][0]

class Concept:
    def __init__(self, name, ontology):
        self.name = name
        self.ontology = ontology
        self.attributes = ontology[self.name]
        self.definition = get_definition(self.name)
        
        self.book = re.findall("^\w*\(.*?\)", self.definition)
        for book in self.book:
            self.definition = self.definition.replace(book, "")
        
        self.attributes[0] = f"definition::{self.definition}"
        # TODO: Fix book

class Question:
    def __init__(self, question_string, answer_list, question_params):
        self.question_string = question_string
        self.answer_list = answer_list
        self.parameters = question_params
        
    def randomise_answers(self):
        correct_answer = self.answer_list[0]
        random.shuffle(self.answer_list)
        return self.answer_list.index(correct_answer)
    
    def print_question_and_answer(self):        
        print(self.question_string)
        self.randomise_answers()
        for answer in self.answer_list:
            print(f"- " + answer.strip("\n"))
        
    def format_output(self):
        self.print_question_and_answer()
        
def get_incorrect_answers(correct_concept, question_parameters):
    # TODO: Move literal strings into a central Constants file so they can be easily changed
    number_of_answers = question_parameters["Answer_number"] if "Answer_number" in question_parameters.keys() else 4
    answer_type = question_parameters["Answer_type"] if "Answer_type" in question_parameters.keys() else None
    
    incorrect_answers = []
    
    while(len(incorrect_answers) < int(number_of_answers) - 1):
        incorrect_concept_name = list(ontology.keys())[random.randint(0, len(ontology))]
        if incorrect_concept_name != correct_concept.name and answer_type == "definition":
            incorrect_answers.append(get_definition(incorrect_concept_name))
        elif incorrect_concept_name != correct_concept.name and answer_type == "concept.name":
            incorrect_answers.append(incorrect_concept_name)
    # TODO: Make answer picker more generalised
    
    return incorrect_answers
    
def get_random_attribute_without_concept_name(concept):
    random_attribute = concept.attributes[random.randint(0, len(concept.attributes) - 1)].replace(concept.name, "_" * 10)
    # print(random_attribute)
    
    if "definition::" in random_attribute:
        random_attribute = f"The definition of {'_' * 10} is: {random_attribute.replace('definition::', '')}"
        
    # return concept.attributes[].replace("{!concept.name}", "_" * 10)
    return random_attribute
    
def replace_template_variables(template_format_string, concept):
    replaced_string = template_format_string
    
    # print(replaced_string)
    
    # get_random_attribute_without_concept_name(concept)
    
    if "{!concept.name}" in replaced_string:
        replaced_string_replacer = get_random_attribute_without_concept_name(concept)
        replaced_string = replaced_string.replace("{!concept.name}", replaced_string_replacer)
    
    # replaced_string = replaced_string.replace("{!concept.name}", "_" * 10)
    replaced_string = replaced_string.replace("{newline}", "\n")
    replaced_string = replaced_string.replace("{concept.name}", concept.name)
    replaced_string = replaced_string.replace("{concept.definition}", concept.definition)
    
    # replaced_string = re.sub("{.*?\!concept\.name.*?}", "__________", replaced_string)
    # replaced_string = re.sub("{.*?newline}.*?", "\n", replaced_string)
    # replaced_string = re.sub("{.*?concept\.name.*?}", concept.name, replaced_string)
    # replaced_string = re.sub("{.*?concept\.definition.*?}", concept.definition, replaced_string)\
    # replaced_string = re.sub("{.*?concept\.relation\.attribute*?}", concept.attribute, replaced_string)
    # replaced_string = re.sub("{.*?concept\.book.*?}", concept.book, replaced_string)
    
    # print(replaced_string)
    
    return replaced_string

def parse_template_question(question_template_to_parse, concept_name="", debug=False):
    template_list = question_template_to_parse.split("::")
    question_parameters = template_list[0].split("(")[1].replace(")", "").split(",")    
    question_parameters = {item.split("=")[0]: item.split("=")[1] for item in question_parameters}
    question = template_list[1].split("->")[1]
    answer = template_list[2].split("->")[1]
            
    if concept_name == "":
        concept_name = list(ontology.keys())[random.randint(0, len(ontology))]     
                
    concept = Concept(concept_name, ontology)
        
    if debug is True:
            print(f"""
{"-" * 100}
Concept:
    Name = {concept.name}
    Definition = {concept.definition}
    Books = {concept.book}
    Attributes = {concept.attributes}
{"-" * 100}
            """)
        
    question = replace_template_variables(question, concept)
    answer = replace_template_variables(answer, concept)
    
    answer_list = [answer]
                
    answer_list = answer_list + get_incorrect_answers(concept, question_parameters)
                
    return Question(question, answer_list, question_parameters)

def parse_template(template_to_parse, concept_name="", debug=False):
    if "question_template" in template_to_parse.split("::")[0]:
        return parse_template_question(template_to_parse, debug=debug)
    else:
        print(f"Error parsing template {template_to_parse}")
        return None
 
parsed_template = parse_template(question_template)

parsed_template.format_output()

# for sentence in sentences:
#     print(nltk.word_tokenize(sentence))
    
# print(nltk.tokenize.TextTilingTokenizer().tokenize(glossary_definitions))
# print(nltk.word_tokenize(glossary_definitions))


Complete the following:
 The definition of __________ is:  A subset of the Change Advisory Board that makes decisions about high-impact Emergency Changes. Membership of the ECAB may be decided at the time a meeting is called, and depends on the nature of the Emergency Change.
- Restoration of Service
- Emergency Change Advisory Board (ECAB)
- Status
- Asset Management


In [ ]:
# for concept in ontology:
#     print(concept)
#     for relationship in ontology[concept]:
#         print(f"\t-{relationship}")




	-definition::(service strategy) a role that is very similar to business relationship manager, but includes more commercial aspects. most commonly used when dealing with external customers.
accounting
	-definition::(service strategy) the process responsible for identifying actual costs of delivering it services, comparing these with budgeted costs, and managing variance from the budget.
	-problem management provides management information about the cost of resolving and preventing problems , which is used as input into the budgeting and accounting systems and total cost of ownership calculations .
	-4.6.7 financial management for it services service operation staff must participate in and support the overall it budgeting and accounting system ï¿½ and may be actively involved in any charging system that may be in place .
	-the service operation manager must also be involved in regular , at least monthly , reviews of expenditure against budgets ï¿½ as part of the ongoing it budgeting an

In [43]:
# # Code from https://spacy.io/
# import spacy
# 
# nlp = spacy.load("en_core_web_sm")
# 
# text = "Event Management is the process that monitors all events that occur through the IT infrastructure to allow for normal operation and also to detect and escalate exception conditions."
# 
# doc = nlp(text)
# 
# # # Analyze syntax
# print("Noun phrases:", [chunk.text for chunk in doc.noun_chunks])
# print("Verbs:", [token.lemma_ for token in doc if token.pos_ == "VERB"])
# 
# print("-" * 100)
# 
# # Find named entities, phrases and concepts
# for entity in doc.ents:
#     print(entity.text, entity.label_)
# 
# print("-" * 100)
# for token in doc:
#     print(token.text, token.pos_, token.dep_)
# 
# print("-" * 100)
# for token in doc:
#     print(token.text, token.dep_)

# from spacy import displacy
# displacy.render(doc, style="dep", jupyter=True, options={'distance': 140})


Noun phrases: ['Event Management', 'the process', 'all events', 'the IT infrastructure', 'normal operation', 'exception conditions']
Verbs: ['be', 'monitor', 'occur', 'allow', 'detect', 'escalate']
----------------------------------------------------------------------------------------------------
Event Management ORG
----------------------------------------------------------------------------------------------------
Event PROPN compound
Management PROPN nsubj
is VERB ROOT
the DET det
process NOUN attr
that DET nsubj
monitors VERB relcl
all DET det
events NOUN dobj
that DET nsubj
occur VERB relcl
through ADP prep
the DET det
IT NOUN compound
infrastructure NOUN pobj
to PART aux
allow VERB advcl
for ADP prep
normal ADJ amod
operation NOUN pobj
and CCONJ cc
also ADV advmod
to PART aux
detect VERB conj
and CCONJ cc
escalate VERB conj
exception NOUN compound
conditions NOUN dobj
. PUNCT punct
--------------------------------------------------------------------------------------------------

In [322]:
# for concept in ontology:
#     print(concept)

# print(ontology["service"][1])

parsed_template = parse_template(question_template, debug=True)
parsed_template.format_output()

# for concept in ontology:
#     # concept = ontology["service"]
#     for index, sentence in enumerate(ontology[concept]):
#         # print(sentence)
#         # print(index)
#         # print(concept)
#         if "definition::" in sentence:
#             continue
#             
#         # print(sentence.replace("service", "{concept}"))
#         ontology[concept][index] = sentence.replace(concept, "{!concept.name}")
#                 
#         # print(sentence.concept.keys()[0])
        
# concept_name = "service"
# print(get_random_attribute_without_concept_name(Concept(concept_name, ontology)))


----------------------------------------------------------------------------------------------------
Concept:
    Name = Tuning
    Definition = The Activity responsible for Planning changes to make the most efficient use of Resources. Tuning is part of Performance Management, which also includes Performance monitoring and implementation of the required Changes.
    Books = []
    Attributes = ['definition::The Activity responsible for Planning changes to make the most efficient use of Resources. Tuning is part of Performance Management, which also includes Performance monitoring and implementation of the required Changes.']
----------------------------------------------------------------------------------------------------
            
Complete the following:
 The definition of __________ is: The Activity responsible for Planning changes to make the most efficient use of Resources. __________ is part of Performance Management, which also includes Performance monitoring and implementa